In [146]:
# Set up dependencies   
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.callbacks import LambdaCallback
import tensorflow as tf
import pandas as pd 

In [147]:
#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [148]:
# Drop the 'EIN' and 'NAME' columns.
colums_to_drop = ["EIN", "NAME"]    
application_df = application_df.drop(colums_to_drop, axis=1)
application_df.head()   

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [149]:
# Determine the number of unique values in each column.
application_df.nunique(axis=0)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [150]:
# Look at APPLICATION_TYPE value counts for binning
application_type = application_df.APPLICATION_TYPE.value_counts()
print(application_type)

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


## Optimization Attempt 1
Change cut off from 250 to 800    

In [151]:
# Cut off at 800 and create a list of application types to be replaced
cutoff_value_app = 800
application_types_to_replace = application_type[application_type < cutoff_value_app].index.tolist()

# Replace with "Other" in the dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, "Other")

# Check to make sure binning was successful
print(application_df['APPLICATION_TYPE'].value_counts())

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64


In [152]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df.CLASSIFICATION.value_counts()
print(classification_counts)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [180]:
# Cut off at 1000 and create a list of classifications to be replaced
cut_off_value_class = 2000  
classifications_to_replace = classification_counts[classification_counts < cut_off_value_class].index.tolist()  

# Replace with "Other" in the dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()  

C1000    17326
C2000     6074
Other     6062
C1200     4837
Name: CLASSIFICATION, dtype: int64

In [181]:
# Check the data types of each column.  
application_df.dtypes

APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [182]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_dummies = pd.get_dummies(application_df)

# Display the resulting DataFrame
application_df_dummies.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,1,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [183]:
# Split the preprocessed data into target and features arrays   
y = application_df_dummies["IS_SUCCESSFUL"].values
X = application_df_dummies.drop(["IS_SUCCESSFUL"], axis=1).values   

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [184]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler with the training data
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [185]:
number_input_features = len(X_train[0])
print(number_input_features)

38


## Optimization attempt 1 

Increase the number of neurons in the input layer

Add two hidden layers 

Increase neurons in each layer 

Change activation functions 

In [186]:
# Define the model
optimize_nn = tf.keras.models.Sequential()  

# Layer 0
optimize_nn.add(tf.keras.layers.Dense(units=128, input_dim=number_input_features, activation='relu'))   

# Layer 1
optimize_nn.add(tf.keras.layers.Dense(units=128, activation='relu'))

# Layer 2
optimize_nn.add(tf.keras.layers.Dense(units=128, activation='tanh'))

# Layer 3
optimize_nn.add(tf.keras.layers.Dense(units=128, activation='relu'))

# Layer 4
optimize_nn.add(tf.keras.layers.Dense(units=128, activation='tanh'))

# Layer 5
optimize_nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
optimize_nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 128)               4992      
                                                                 
 dense_17 (Dense)            (None, 128)               16512     
                                                                 
 dense_18 (Dense)            (None, 128)               16512     
                                                                 
 dense_19 (Dense)            (None, 128)               16512     
                                                                 
 dense_20 (Dense)            (None, 128)               16512     
                                                                 
 dense_21 (Dense)            (None, 1)                 129       
                                                                 
Total params: 71169 (278.00 KB)
Trainable params: 7116

In [187]:
# Compile the model
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)
optimize_nn.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [188]:
# Train the model
optimize_nn.fit(X_train_scaled, y_train, epochs=100, callbacks=None)

Epoch 1/100
804/804 [==============================] - 1s 905us/step - loss: 0.5767 - accuracy: 0.7194
Epoch 2/100
804/804 [==============================] - 1s 930us/step - loss: 0.5642 - accuracy: 0.7265
Epoch 3/100
804/804 [==============================] - 1s 995us/step - loss: 0.5609 - accuracy: 0.7286
Epoch 4/100
804/804 [==============================] - 1s 875us/step - loss: 0.5589 - accuracy: 0.7308
Epoch 5/100
804/804 [==============================] - 1s 909us/step - loss: 0.5578 - accuracy: 0.7310
Epoch 6/100
804/804 [==============================] - 1s 877us/step - loss: 0.5571 - accuracy: 0.7318
Epoch 7/100
804/804 [==============================] - 1s 880us/step - loss: 0.5559 - accuracy: 0.7318
Epoch 8/100
804/804 [==============================] - 1s 915us/step - loss: 0.5551 - accuracy: 0.7317
Epoch 9/100
804/804 [==============================] - 1s 864us/step - loss: 0.5551 - accuracy: 0.7304
Epoch 10/100
804/804 [==============================] - 1s 870us/step - l

In [191]:
# Evaluate the model using the test data    
model_loss, model_accuracy = optimize_nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5682 - accuracy: 0.7251 - 116ms/epoch - 431us/step
Loss: 0.568240761756897, Accuracy: 0.7251312136650085


## Attempt 2
Increase the number of neurons in the hidden layers 

Increase the number of epochs 

Use relu for the hidden layers and sigmoid for the output layer  

In [192]:
# Define the model 
optimize_nn2 = tf.keras.models.Sequential()  

# Layer 0
optimize_nn2.add(tf.keras.layers.Dense(units=200, input_dim=number_input_features, activation='relu'))   

# Layer 1
optimize_nn2.add(tf.keras.layers.Dense(units=200, activation='relu'))

# Layer 2
optimize_nn2.add(tf.keras.layers.Dense(units=200, activation='relu'))

# Layer 3 
optimize_nn2.add(tf.keras.layers.Dense(units=200, activation='relu'))

# Layer 4
optimize_nn.add(tf.keras.layers.Dense(units=200, activation='relu'))

# Layer 5
optimize_nn2.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
optimize_nn2.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_22 (Dense)            (None, 200)               7800      
                                                                 
 dense_23 (Dense)            (None, 200)               40200     
                                                                 
 dense_24 (Dense)            (None, 200)               40200     
                                                                 
 dense_25 (Dense)            (None, 200)               40200     
                                                                 
 dense_27 (Dense)            (None, 1)                 201       
                                                                 
Total params: 128601 (502.35 KB)
Trainable params: 128601 (502.35 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [193]:
# Compile the model
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)
optimize_nn2.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [194]:
# Train the model
optimize_nn2.fit(X_train_scaled, y_train, epochs=200, callbacks=None)

Epoch 1/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5740 - accuracy: 0.7231
Epoch 2/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5629 - accuracy: 0.7276
Epoch 3/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5607 - accuracy: 0.7302
Epoch 4/200
804/804 [==============================] - 1s 962us/step - loss: 0.5589 - accuracy: 0.7312
Epoch 5/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5575 - accuracy: 0.7313
Epoch 6/200
804/804 [==============================] - 1s 990us/step - loss: 0.5564 - accuracy: 0.7310
Epoch 7/200
804/804 [==============================] - 1s 976us/step - loss: 0.5557 - accuracy: 0.7320
Epoch 8/200
804/804 [==============================] - 1s 987us/step - loss: 0.5546 - accuracy: 0.7322
Epoch 9/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5545 - accuracy: 0.7321
Epoch 10/200
804/804 [==============================] - 1s 978us/step - loss: 0.554

In [195]:
# Evaluate the model using the test data    
model_loss, model_accuracy = optimize_nn2.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.8017 - accuracy: 0.7244 - 191ms/epoch - 714us/step
Loss: 0.8017259836196899, Accuracy: 0.7244315147399902


## Use keras tuner to try to find the best hyperparameters for the model   

In [197]:
# Set up keras tuner
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch


def build_model(hp):
    tuner_model = keras.Sequential()

    # Choose the number of layers
    for i in range(hp.Int('num_layers', min_value=1, max_value=5)):
        # Choose the number of neurons in each layer
        tuner_model.add(layers.Dense(units=hp.Int(f'units_{i}', min_value=32, max_value=512, step=32), 
                                     activation=hp.Choice(f'activation_{i}', ['relu', 'tanh', 'elu'])))
    
    tuner_model.add(layers.Dense(1, activation='sigmoid'))

    tuner_model.compile(optimizer=keras.optimizers.legacy.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return tuner_model

# Initialize the tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    project_name='Tuner_hypers' 
)

# Search for the best hyperparameter configuration
tuner.search(X_train_scaled, y_train, epochs=100, validation_data=(X_train_scaled, y_train))

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the final model with the best hyperparameters
final_model = tuner.hypermodel.build(best_hps)

Trial 10 Complete [00h 01m 34s]
val_accuracy: 0.7395817041397095

Best val_accuracy So Far: 0.7395817041397095
Total elapsed time: 00h 24m 55s


In [198]:
# Print the best hyperparameters
print("Best Hyperparameters:")
print(f"Number of Layers: {best_hps.get('num_layers')}")
for i in range(best_hps.get('num_layers')):
    print(f"Units in Layer {i}: {best_hps.get(f'units_{i}')}")
    print(f"Activation in Layer {i}: {best_hps.get(f'activation_{i}')}")
print(f"Learning Rate: {best_hps.get('learning_rate')}")

Best Hyperparameters:
Number of Layers: 3
Units in Layer 0: 32
Activation in Layer 0: tanh
Units in Layer 1: 192
Activation in Layer 1: tanh
Units in Layer 2: 256
Activation in Layer 2: tanh
Learning Rate: 0.001


## Attempt 3
Use the results from the RandomSearch to build a new model with the best hyperparameters.    

In [203]:
# Define the model
optimize_nn3 = tf.keras.models.Sequential()  

# Layer 0
optimize_nn3.add(tf.keras.layers.Dense(units=32, input_dim=number_input_features, activation='tanh'))   

# Layer 1
optimize_nn3.add(tf.keras.layers.Dense(units=192, activation='tanh'))

# Layer 2
optimize_nn3.add(tf.keras.layers.Dense(units=256, activation='tanh'))

# Output layer 
optimize_nn3.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model

optimize_nn3.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 32)                1248      
                                                                 
 dense_17 (Dense)            (None, 192)               6336      
                                                                 
 dense_18 (Dense)            (None, 256)               49408     
                                                                 
 dense_19 (Dense)            (None, 1)                 257       
                                                                 
Total params: 57249 (223.63 KB)
Trainable params: 57249 (223.63 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [204]:
# Compile the model
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)
optimize_nn3.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy']) 

In [205]:
# Train the model
optimize_nn3.fit(X_train_scaled, y_train, epochs=100, callbacks=None)   

Epoch 1/100
804/804 [==============================] - 1s 797us/step - loss: 0.5902 - accuracy: 0.7151
Epoch 2/100
804/804 [==============================] - 1s 829us/step - loss: 0.5714 - accuracy: 0.7240
Epoch 3/100
804/804 [==============================] - 1s 779us/step - loss: 0.5656 - accuracy: 0.7275
Epoch 4/100
804/804 [==============================] - 1s 793us/step - loss: 0.5628 - accuracy: 0.7289
Epoch 5/100
804/804 [==============================] - 1s 768us/step - loss: 0.5608 - accuracy: 0.7293
Epoch 6/100
804/804 [==============================] - 1s 784us/step - loss: 0.5590 - accuracy: 0.7294
Epoch 7/100
804/804 [==============================] - 1s 826us/step - loss: 0.5581 - accuracy: 0.7291
Epoch 8/100
804/804 [==============================] - 1s 784us/step - loss: 0.5571 - accuracy: 0.7307
Epoch 9/100
804/804 [==============================] - 1s 776us/step - loss: 0.5564 - accuracy: 0.7305
Epoch 10/100
804/804 [==============================] - 1s 784us/step - l

In [206]:
# # Evaluate the model using the test data
model_loss, model_accuracy = optimize_nn3.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5657 - accuracy: 0.7163 - 159ms/epoch - 592us/step
Loss: 0.5657157897949219, Accuracy: 0.7162682414054871
